In [ ]:
import sys
sys.path.append('.')

import common
from sklearn import tree
import os


root_dir = "raw_data_2023_0719"
raw_data_perfix = "2023_07_21_all"

raw_data_file_path = os.path.join(root_dir, raw_data_perfix + ".csv")
raw_time_splits_path = os.path.join(root_dir, raw_data_perfix + ".json")

X, Y, feature_names, class_names, apps = common.read_all_in_one_data(raw_data_file_path, raw_time_splits_path)

In [ ]:
x_train, x_test, y_train, y_test = common.spilt_for_train_test(X, Y, mode="class", apps=apps, app="redis")
# x_train, x_test, y_train, y_test = common.spilt_for_train_test(X, Y, mode="random")

In [ ]:
# model optimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

cv = 3
clf = tree.DecisionTreeClassifier()
parameters = {'criterion': ['gini', 'entropy'],
              'max_depth': [3, 5, 7],
              'min_samples_split': [2, 4, 6],
              'min_samples_leaf': [1, 3, 5, 7]}

# scores = cross_val_score(clf, X, Y, cv=cv, scoring="f1_macro")
# for i in range(0, 32):
grid_search = GridSearchCV(clf, parameters, cv=cv, scoring='f1_macro')
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

common.model_score(best_model, x_test, y_test)
common.save_dt_to_png(f"best_tree", best_model, feature_names, class_names)

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf = clf.fit(common.normalize(x_train), y_train)

common.model_score(clf, common.normalize(x_test), y_test)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

random_state = 170

common_params = {
    "n_init": "auto",
    "random_state": random_state,
}

y_pred = KMeans(n_clusters=8, **common_params).fit_predict(X)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
axs[0, 0].scatter(X[:, 1], X[:, 2], c=y_pred)
axs[0, 0].set_title("Non-optimal Number of Clusters")

axs[0, 1].scatter(X[:, 1], X[:, 3], c=y_pred)
axs[0, 1].set_title("Non-optimal Number of Clusters")

axs[1, 0].scatter(X[:, 1], X[:, 4], c=y_pred)
axs[1, 0].set_title("Non-optimal Number of Clusters")

axs[1, 1].scatter(X[:, 1], X[:, 5], c=y_pred)
axs[1, 1].set_title("Non-optimal Number of Clusters")